In [15]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

## Read CSV

In [16]:
aa_csv = []
anka_csv = []
for csv in os.listdir("raw_csv"):
    if "aa_" in csv:
        aa_csv.append(csv)
    elif "anka_" in csv:
        anka_csv.append(csv)

In [17]:
aa_df_list = [pd.read_csv(f"raw_csv/{file}") for file in aa_csv]
anka_df_list = [pd.read_csv(f"raw_csv/{file}") for file in anka_csv]

In [18]:
aa_df = pd.concat(aa_df_list, ignore_index=True)
anka_df = pd.concat(anka_df_list, ignore_index=True)

In [19]:
aa_df = aa_df.drop_duplicates()
anka_df = anka_df.drop_duplicates() 

In [20]:
aa_df["All_Title"] = aa_df["Başlık"] + ". " + aa_df["Alt Başlık"]

In [21]:
csv_list = []
for title in aa_df.to_dict()["All_Title"].values():
    csv_list.append({"text": f"{title}", "source": ("Title","AA")})
for text in aa_df.to_dict()["Makale"].values():
    csv_list.append({"text": f"{text}", "source": ("Text", "AA")})

In [22]:
for title in anka_df.to_dict()["Başlık"].values():
    csv_list.append({"text": f"{title}", "source": ("Title","ANKA")})
for text in anka_df.to_dict()["Makale"].values():
    csv_list.append({"text": f"{text}", "source": ("Text", "ANKA")})

## Clear
1) Cid
2) non-utf8
3) Analyze All Unique Character

In [9]:
txt_list = []
folder_list = os.listdir('raw_extracted')
for folder in folder_list:
    print(folder)
    txt_files = os.listdir(f'raw_extracted/{folder}')
    for txt_file in txt_files:
        try:
            with open(f'raw_extracted/{folder}/{txt_file}', encoding = 'utf-8') as file:
                for txt in file.readlines():
                    if "cid:" in txt:
                        continue
                    else:
                        txt_list.append({'text': txt, 'source': (txt_file, folder)})
        except UnicodeDecodeError:
            continue

DergiPark_TXT
PKF_TXT
TSBP_TXT
YOK_TEXT


In [23]:
all_txt = txt_list + csv_list 

In [24]:
len(all_txt)

3534327

In [25]:
max_len = 2000

def split_text(text, max_len):
    return [text[i:i + max_len] for i in range(0, len(text), max_len)]

all_txt_split = []

for entry in all_txt:
    if len(entry['text']) > max_len:
        split_parts = split_text(entry['text'], max_len)
        for part in split_parts:
            all_txt_split.append({'text': part, 'source': entry['source']})
    else:
        all_txt_split.append(entry)




In [26]:
len(all_txt_split)

5780871

In [27]:
def clean_string(input_string):
    allowed_chars = set([
        'A', 'a', 'B', 'b', 'C', 'c', 'Ç', 'ç', 'D', 'd', 'E', 'e', 'F', 'f', 'G', 'g',
        'Ğ', 'ğ', 'H', 'h', 'I', 'ı', 'İ', 'i', 'J', 'j', 'K', 'k', 'L', 'l', 'M', 'm',
        'N', 'n', 'O', 'o', 'Ö', 'ö', 'P', 'p', 'R', 'r', 'S', 's', 'Ş', 'ş', 'T', 't',
        'U', 'u', 'Ü', 'ü', 'V', 'v', 'Y', 'y', 'Z', 'z',
        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
        '.', ',', ':', ';', '!', '?', '"', "'", '-', '—', '(', ')', '[', ']', '{', '}',
        '/', '\\', '*', '%', '@', '#', '$', '&', '^', '|', '~', ' ',
        '+', '=', '<', '>'
    ])

    return ''.join(filter(allowed_chars.__contains__, input_string))


In [28]:
for index in range(len(txt_list)):
    all_txt_split[index]["text"] = clean_string(txt_list[index]["text"])

In [29]:
df = pd.DataFrame(all_txt_split)
#df.to_csv("dataset_law.csv")

In [30]:
# Train Test Split
train, val = train_test_split(df, test_size = 0.010, random_state=42)

# Save in parquet for less memory

In [ ]:
df.to_parquet("dataset_finance.parquet",
              engine = "pyarrow",
              compression="snappy",
              index = False)

In [ ]:
train.to_parquet("train_finance.parquet",
              engine = "pyarrow",
              compression="snappy",
              index = False)

In [ ]:
val.to_parquet("val_finance.parquet",
              engine = "pyarrow",
              compression="snappy",
              index = False)